In [21]:
import sys
import pandas as pd
sys.path.append("..")
from analysis.utils.connection import get_conn


In [65]:
def load_sales():
    conn = get_conn()
    query = """ 
    Select ProductKey, ProductName, ABC_Class, TotalRevenue, CumulativeRevenuePct 
    From dbo.vw_ProductABC
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df
abc_sales = load_sales()
display(abc_sales.head())
display(abc_sales.tail())
    

C:\Users\athil\AppData\Local\Temp\ipykernel_12180\469199524.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ProductKey,ProductName,ABC_Class,TotalRevenue,CumulativeRevenuePct
0,1,Contoso 512MB MP3 Player E51 Silver,C,844.35,0.997990
1,2,Contoso 512MB MP3 Player E51 Blue,C,1000.23,0.997363
2,3,Contoso 1G MP3 Player E100 White,C,1161.60,0.996771
3,4,Contoso 2G MP3 Player E200 Silver,C,1596.18,0.994644
4,5,Contoso 2G MP3 Player E200 Red,C,1056.93,0.997132


,ProductKey,ProductName,ABC_Class,TotalRevenue,CumulativeRevenuePct
2487,2513,Contoso Bluetooth Active Headphones L15 Red,B,21188.37,0.809570
2488,2514,Contoso Bluetooth Active Headphones L15 White,A,27427.89,0.750061
2489,2515,Contoso In-Line Coupler E180 White,C,596.30,0.998927
2490,2516,Contoso In-Line Coupler E180 Black,C,522.60,0.999169
2491,2517,Contoso In-Line Coupler E180 Silver,C,586.25,0.998968


In [58]:
def load_sales_transactions():
    conn = get_conn()
    query = """
    SELECT CustomerKey, ProductKey, TotalSalesUSD FROM dbo.FactSales
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df
sales_transactions = load_sales_transactions()
display(sales_transactions.head())

C:\Users\athil\AppData\Local\Temp\ipykernel_12180\2553189192.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,CustomerKey,ProductKey,TotalSalesUSD
0,276108,30,1530.00
1,979530,1715,350.65
2,979530,1556,280.00
3,979530,2129,5221.93
4,979530,28,799.60


In [63]:
def load_at_risk():
    conn = get_conn()
    query = """
    SELECT CustomerKey
        FROM dbo.rfm_customer_segments
        WHERE Segment = 'At Risk'
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df
at_risk = load_at_risk()
display(at_risk.head())

C:\Users\athil\AppData\Local\Temp\ipykernel_12180\3679111126.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,CustomerKey
0,554
1,1314
2,2238
3,2435
4,5962


In [69]:
sales_at_risk = sales_transactions.merge(
    at_risk,
    on = "CustomerKey",
    how = "inner"
)
display(sales_at_risk.head())

,CustomerKey,ProductKey,TotalSalesUSD
0,1559404,1284,99.96
1,1559404,1431,512.00
2,1559404,2506,14.22
3,1559404,1354,57.98
4,1559404,1439,602.00


In [73]:
sales_at_risk_abc = sales_at_risk.merge(
    abc_sales,
    on = "ProductKey",
    how = "left"
)
display(sales_at_risk_abc.head())

,CustomerKey,ProductKey,TotalSalesUSD,ProductName,ABC_Class,TotalRevenue,CumulativeRevenuePct
0,1559404,1284,99.96,Contoso Mini Battery Charger Kit E320 Black,C,1949.22,0.992662
1,1559404,1431,512.00,The Phone Company Touch Screen Phones 5-Wire/B...,A,43520.00,0.602756
2,1559404,2506,14.22,Contoso Rubberized Snap-On Cover Hard Case Cel...,C,853.20,0.997917
3,1559404,1354,57.98,Contoso Hybrid system M60 White,C,1681.42,0.994364
4,1559404,1439,602.00,The Phone Company Sharp Touch Screen Phones M9...,A,57190.00,0.517410


In [79]:
exposure = ( 
    sales_at_risk_abc
    .groupby("ABC_Class", as_index = False)
    .agg(Revenue_From_At_Risk = ("TotalSalesUSD", "sum"))
)
display(exposure.head())

,ABC_Class,Revenue_From_At_Risk
0,A,3610822.78
1,B,873922.88
2,C,274165.59


In [81]:
total_revenue_abc = (
    sales_transactions.merge(abc_sales, on="ProductKey",how="left")
    .groupby("ABC_Class", as_index = False)
    .agg(
        Total_Revenue=("TotalSalesUSD", "sum")
    )
)
display(total_revenue_abc.head())

,ABC_Class,Total_Revenue
0,A,46552808.24
1,B,8735646.07
2,C,2912526.46


In [87]:
exposure_final = exposure.merge(
    total_revenue_abc,
    on="ABC_Class",
    how="left"
)

exposure_final["Pct_Revenue_At_Risk"] = (
    exposure_final["Revenue_From_At_Risk"]
    / exposure_final["Total_Revenue"]
)
display(exposure_final.head())

,ABC_Class,Revenue_From_At_Risk,Total_Revenue,Pct_Revenue_At_Risk
0,A,3610822.78,46552808.24,0.077564
1,B,873922.88,8735646.07,0.100041
2,C,274165.59,2912526.46,0.094133
